In [1]:
import json

from pathlib import PurePath
from glob import glob
from tqdm.auto import tqdm

In [2]:
# спарсим данные по каналам

channels = []
for fl_name, area in tqdm([('data/support/channels.json', 'support'), ('data/team/channels.json', 'team')]):
    with open(fl_name, 'r') as fl:
        js_channel = json.load(fl)
    for channel in js_channel:
        channels.append(
            {
                'channel_id': channel['id'],
                'name': channel['name'].lower(),
                'is_archived': channel['is_archived'],
                'members': ' '.join(channel['members']).lower(),
                'topic': channel['topic']['value'].lower(),
                'desc': channel['purpose']['value'].lower(),
                'area': area.lower()
            }
        )

In [8]:
# спрасим пользователей

users = []
for fl_name, area in tqdm([('data/support/users.json', 'support'), ('data/team/users.json', 'team')]):
    with open(fl_name, 'r') as fl:
        js_user = json.load(fl)
        for user in js_user:
            email = user['profile'].get('email', None)
            email = email.lower() if email else email
            users.append(
                {
                    'user_id': user['id'],
                    'deleted': user['deleted'],
                    'real_name': user['profile']['real_name'],
                    'display_name': user['profile']['display_name'],
                    'email': email,
                    'tz_offset': user.get('tz_offset', None),
                    'area': area
                }
            )
                
                
         

In [10]:
# спарсим сообщения
messages = []

for pattern_file_name, area in [('data/support/*/*.json', 'support'), ('data/team/*/*.json', 'team')]:
    for file_name in tqdm(glob(pattern_file_name)):
        with open(file_name, 'r') as fl:
            for message in filter(lambda x: 'user_profile' in x.keys(), json.load(fl)):
                messages.append(
                    {
                        'text': message['text'].lower(),
                        'user_id': message['user'],
                        'ts': str(message['ts']),
                        'area': area.lower(),
                        'channel': PurePath(file_name).parts[-2]
                    }
                )

In [11]:
for file_name, data in [('users.json', users), ('channels.json', channels), ('messages.json', messages)]:
    with open(f"new_data/{file_name}", 'w') as fl:
        json.dump(data, fl)